In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
data_path = "drive/My Drive/_data/"
!ls "drive/My Drive/"
!ls "drive/My Drive/_data"




import sys
sys.path.append('drive/My Drive/')

import pandas as pd

import unicodedata
import re

import nltk
nltk.download('treebank')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import word_tokenize


import os
import numpy as np
from keras.models import Sequential, Model
from keras.layers import *#Embedding,Bidirectional,Dense,Conv1D,Flatten,LSTM,GlobalMaxPooling1D,Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import pickle

In [58]:
data_file = data_path + "valid.tsv"
df_v = pd.read_csv(data_file, header=None, sep='\t', usecols=[1,2], names=['Label', 'Statement'])
df_v = df_v.dropna() 

data_file = data_path + "test.tsv"
df_t = pd.read_csv(data_file, header=None, sep='\t', usecols=[1,2], names=['Label', 'Statement'])
df_t = df_t.dropna()

data_file = data_path + "train.tsv"
df = pd.read_csv(data_file, header=None, sep='\t', usecols=[1,2], names=['Label', 'Statement'])
df = df.dropna()

print(len(df))
print(len(df.columns))
print(df.columns)
print(df.Label.unique())
df.head()
# df['Statement'][0]

10240
2
Index(['Label', 'Statement'], dtype='object')
['false' 'half-true' 'mostly-true' 'true' 'barely-true' 'pants-fire']


,Label,Statement
0,false,Says the Annies List political group supports ...
1,half-true,When did the decline of coal start? It started...
2,mostly-true,"Hillary Clinton agrees with John McCain ""by vo..."
3,false,Health care reform legislation is likely to ma...
4,half-true,The economic turnaround started at the end of ...


#preprocess

In [5]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation_number(words):
    """Remove punctuation & number from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'([^\s\w]|_|[0-9])+', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

stop_words = set(stopwords.words("english"))
def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation_number(words)
    # words = remove_stopwords(words)
    # words = stem_words(words)
    # words = lemmatize_verbs(words)
    return words

def norm_all_sent(_sentences):
  for _i,_w in enumerate(_sentences):
    # print(_i,_w)
    tmp_sentences = _w#_sentences[_w]
    tmp_sentences = word_tokenize(tmp_sentences)
    tmp_sentences = normalize(tmp_sentences)
    tmp_sentences = " ".join(tmp_sentences)
    _sentences[_i] = tmp_sentences
  return _sentences

def convert_labels(_label):
#['false' 'half-true' 'mostly-true' 'true' 'barely-true' 'pants-fire']  
    _label[_label=='false']=0
    _label[_label=='pants-fire']=1
    _label[_label=='half-true']=2
    _label[_label=='mostly-true']=3
    _label[_label=='barely-true']=4
    _label[_label=='true']=5
    return _label

In [6]:
embed_num_dims = 100
max_seq_len = 100

sentences = df['Statement'].values
sentences = norm_all_sent(sentences)
Y = df['Label'].values
Y = convert_labels(Y).astype(np.int)


sentences_t = df_t['Statement'].values
sentences_t = norm_all_sent(sentences_t)
Y_t = df_t['Label'].values
Y_t = convert_labels(Y_t).astype(np.int)


sentences_v = df_v['Statement'].values
sentences_v = norm_all_sent(sentences_v)
Y_v = df_v['Label'].values
Y_v = convert_labels(Y_v).astype(np.int)


tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
sequence = tokenizer.texts_to_sequences(sentences)
sequence_t = tokenizer.texts_to_sequences(sentences_t)
sequence_v = tokenizer.texts_to_sequences(sentences_v)

index_of_words = tokenizer.word_index
print("len(index_of_words)",len(index_of_words))

padded_seq = pad_sequences(sequence , maxlen = max_seq_len )
padded_seq_t = pad_sequences(sequence_t , maxlen = max_seq_len )
padded_seq_v = pad_sequences(sequence_v , maxlen = max_seq_len )

from keras.utils import to_categorical
Y = to_categorical(Y)
Y_t = to_categorical(Y_t)
Y_v = to_categorical(Y_v)

X_train=padded_seq
Y_train=Y
X_test=padded_seq_t
Y_test=Y_t
X_val=padded_seq_v
Y_val=Y_v



len(index_of_words) 12331


#CredRank Algorithm
##Measuring User Credibility in Social Media

In [28]:
try:
    import flair
except:
    !pip install flair
import pickle
from flair.embeddings import TransformerDocumentEmbeddings
from flair.data import Sentence
from IPython.display import clear_output


selected_transformer_model="bert-base-cased"
embedding = TransformerDocumentEmbeddings(selected_transformer_model)

def sentence_embedding(input_S):
    _input = Sentence(input_S)
    embedding.embed(_input)
    _input.get_embedding()
    _emb = _input.get_embedding()
    _emb = _emb.detach().cpu().numpy()
    return _emb


In [30]:
# #########Train##############
# all_emb = []
# for _i,_sen in enumerate(df['Statement']):
#     x = _sen
#     _emb = sentence_embedding(x)
#     print(_i,'=>',_emb.shape)
#     all_emb.append(_emb)


# outfile = data_path + "train_new.npy"
# all_emb = np.array(all_emb)
# np.save(outfile, all_emb)
# clear_output()

# #########Test##############
# all_emb = []
# for _i,_sen in enumerate(df_t['Statement']):
#     x = _sen
#     _emb = sentence_embedding(x)
#     print(_i,'=>',_emb.shape)
#     all_emb.append(_emb)


# outfile = data_path + "test_new.npy"
# all_emb = np.array(all_emb)
# np.save(outfile, all_emb)

# #########Valid##############
# all_emb = []
# for _i,_sen in enumerate(df_v['Statement']):
#     x = _sen
#     _emb = sentence_embedding(x)
#     print(_i,'=>',_emb.shape)
#     all_emb.append(_emb)


# outfile = data_path + "valid_new.npy"
# all_emb = np.array(all_emb)
# np.save(outfile, all_emb)

# clear_output()


'the department of veterans affairs has a manual out there telling our veterans stuff like are you really of value to your community you know encouraging them to commit suicide'

In [7]:
data_file = data_path + "train_new.npy"
all_emb = np.load(data_file)
print("all_emb_train.shape=>",all_emb.shape)

data_file = data_path + "test_new.npy"
all_emb_t = np.load(data_file)
print("all_emb_test.shape=>",all_emb_t.shape)

data_file = data_path + "valid_new.npy"
all_emb_v = np.load(data_file)
print("all_emb_valid.shape=>",all_emb_v.shape)


tmp_all_emb = np.concatenate((all_emb,all_emb_t,all_emb_v))
print("tmp_all_emb.shape=>",tmp_all_emb.shape)

all_emb_train.shape=> (10240, 768)
all_emb_test.shape=> (1267, 768)
all_emb_valid.shape=> (1284, 768)
tmp_all_emb.shape=> (12791, 768)


In [9]:
# # # # from scipy.cluster.hierarchy import dendrogram, linkage
# # # # from matplotlib import pyplot as plt

# # # # linked = linkage(all_emb_t[:10,:], 'single')

# # # # plt.figure(figsize=(10, 7))
# # # # dendrogram (linked,
# # # #             orientation='top',
# # # #             distance_sort='descending',
# # # #             show_leaf_counts=True)
# # # # plt.show()


from sklearn.cluster import AgglomerativeClustering
cluster = AgglomerativeClustering(n_clusters=5,affinity='euclidean', linkage='ward')# affinity="cosine", linkage='average')
tmp_c=cluster.fit_predict(tmp_all_emb)

outfile = data_path + "clustering_result_new.npy"
np.save(outfile, tmp_c)

In [10]:
from collections import Counter

data_file = data_path + "clustering_result_new.npy"
tmp_c = np.load(data_file)


train_c = tmp_c[:len(all_emb)]
print(len(train_c))
print(np.unique(train_c))
print(Counter(train_c))
# print(Counter(Y_train.argmax(axis=1)))

test_c = tmp_c[len(all_emb):len(all_emb)+len(all_emb_t)]
print(len(test_c))
print(np.unique(test_c))
print(Counter(test_c))
# print(Counter(Y_train.argmax(axis=1)))

valid_c = tmp_c[len(all_emb)+len(all_emb_t):]
print(len(valid_c))
print(np.unique(valid_c))
print(Counter(valid_c))


n_of_m = Counter(train_c)
cluster_weight = np.zeros(len(n_of_m))
_tmp = np.array(list(n_of_m.values()))
_for_div = np.sum(np.sqrt(_tmp))
for _cluster in n_of_m:
    _c_i = np.sqrt(n_of_m[_cluster])
    # member_weight = np.sqrt(n_of_m[_cluster])/n_of_m[_cluster]
    cluster_weight[_cluster] = (_c_i/_for_div)#*member_weight


10240
[0 1 2 3 4]
Counter({0: 3859, 1: 2979, 2: 1652, 4: 1601, 3: 149})
1267
[0 1 2 3 4]
Counter({0: 467, 1: 365, 2: 220, 4: 192, 3: 23})
1284
[0 1 2 3 4]
Counter({0: 514, 1: 365, 2: 200, 4: 193, 3: 12})


#Word Embedding

In [11]:
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !unzip glove*.zip
# !mv glove*.100d.txt 'drive/My Drive/_MrAnsari/_data'

f = open(data_path+'glove.6B.100d.txt')
embedd_index = {}
for line in f:
    val = line.split()
    word = val[0]
    coff = np.asarray(val[1:],dtype = 'float')
    embedd_index[word] = coff

f.close()
print('Found %s word vectors.' % len(embedd_index))

# a_file = open(data_path+"data.pkl", "wb")
# pickle.dump(embedd_index, a_file)
# a_file.close()

Found 400000 word vectors.


In [ ]:
# a_file = open(data_path+"data.pkl", "rb")
# output = pickle.load(a_file)
# print('Found %s word vectors.' % len(embedd_index))

embedd_index['good']

In [12]:
embedding_matrix = np.zeros((len(index_of_words) + 1, embed_num_dims))

tokens = []
labels = []

for word,i in index_of_words.items():
    temp = embedd_index.get(word)
    if temp is not None:
        embedding_matrix[i] = temp
        
#for plotting
        tokens.append(embedding_matrix[i])
        labels.append(word)

print(embedding_matrix.shape)

(12332, 100)


In [ ]:
#TSNE algorithm used to visualize word embeddings having huge amount (100) dimensions

def tsne():
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens[:200])
    print(new_values.shape)
    
    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(16,16)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()

    
    
    
    
tsne()

In [73]:
from keras.optimizers import Adam

input1 = Input(shape=(max_seq_len,))
input2 = Input(shape=(1,))

#Embedding layer before the actaul BLSTM 
embedd_layer = Embedding(len(index_of_words) + 1 , embed_num_dims , input_length = max_seq_len , weights = [embedding_matrix])(input1)
BiLSTM = Bidirectional(LSTM(64 , return_sequences = True , dropout = 0.2 , recurrent_dropout = 0.2))(embedd_layer)
GMP = GlobalMaxPooling1D()(BiLSTM)
FC1 = Dense(32,activation = 'relu')(GMP)
FC1_d = Dropout(0.2)(FC1)
FC2 = Dense(6,activation = 'sigmoid')(FC1_d)
_newinput = Concatenate()([FC2, input2])
FC3 = Dense(6,activation = 'softmax')(_newinput)
model = Model(inputs=[input1, input2], outputs=FC3)
model.summary()

_adam = Adam(lr = 0.01)
model.compile(loss = 'categorical_crossentropy' , optimizer = _adam , metrics = ['accuracy'])


Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 100, 100)     1233200     input_7[0][0]                    
__________________________________________________________________________________________________
bidirectional_3 (Bidirectional) (None, 100, 128)     84480       embedding_3[0][0]                
__________________________________________________________________________________________________
global_max_pooling1d_3 (GlobalM (None, 128)          0           bidirectional_3[0][0]            
_______________________________________________________________________________________

In [74]:
new_data = []
for _i in range(len(train_c)):
    tmp_data = cluster_weight[train_c[_i]]
    new_data.append(tmp_data)
new_data = np.array(new_data)

new_data_t = []
for _i in range(len(test_c)):
    tmp_data = cluster_weight[test_c[_i]]
    new_data_t.append(tmp_data)
new_data_t = np.array(new_data_t)

new_data_v = []
for _i in range(len(valid_c)):
    tmp_data = cluster_weight[valid_c[_i]]
    new_data_v.append(tmp_data)
new_data_v = np.array(new_data_v)



In [75]:
hist = model.fit([X_train,new_data],Y_train,epochs = 50 , batch_size = 256, validation_data = ([X_val,new_data_v],Y_val))

result = model.evaluate([X_test,new_data_t],Y_test)
print(result[1]*100)

model_file = data_path + "LSTM_Model6_new.h5"
model.save(model_file)

Epoch 1/5
40/40 [==============================] - 17s 418ms/step - loss: 1.7704 - accuracy: 0.2036 - val_loss: 1.7666 - val_accuracy: 0.1931
Epoch 2/5
40/40 [==============================] - 16s 406ms/step - loss: 1.7578 - accuracy: 0.2062 - val_loss: 1.7627 - val_accuracy: 0.1970
Epoch 3/5
40/40 [==============================] - 16s 409ms/step - loss: 1.7577 - accuracy: 0.2066 - val_loss: 1.7662 - val_accuracy: 0.1931
Epoch 4/5
40/40 [==============================] - 16s 406ms/step - loss: 1.7580 - accuracy: 0.2089 - val_loss: 1.7650 - val_accuracy: 0.1908
Epoch 5/5
40/40 [==============================] - 3s 73ms/step - loss: 1.7501 - accuracy: 0.2115
21.152327954769135


In [76]:
model_file = data_path + "LSTM_Model6_new.h5"
model.load_weights(model_file)

result = model.evaluate([X_test,new_data_t],Y_test)
print(result[1]*100)


40/40 [==============================] - 3s 84ms/step - loss: 1.7501 - accuracy: 0.2115
21.152327954769135


In [77]:
#######################################
new_data = []
for _i in range(len(train_c)):
    tmp_data = cluster_weight[train_c[_i]]
    new_data.append(tmp_data)
new_data = np.array(new_data)

new_data_t = []
for _i in range(len(test_c)):
    tmp_data = cluster_weight[test_c[_i]]
    new_data_t.append(tmp_data)
new_data_t = np.array(new_data_t)

new_data_v = []
for _i in range(len(valid_c)):
    tmp_data = cluster_weight[valid_c[_i]]
    new_data_v.append(tmp_data)
new_data_v = np.array(new_data_v)
#######################################
#######################################

y_pred_v  = model.predict([X_val,new_data_v])
# print("Valid_Accuracy=>",np.mean((y_pred_v.argmax(axis=1)==Y_val.argmax(axis=1))))

y_pred_t  = model.predict([X_test,new_data_t])
# print("Test_Accuracy=>",np.mean((y_pred_t.argmax(axis=1)==Y_test.argmax(axis=1))))

y_pred  = model.predict([X_train,new_data])
# print("Train_Accuracy=>",np.mean((y_pred.argmax(axis=1)==Y_train.argmax(axis=1))))

new_data = []
for _i in range(y_pred.shape[0]):
    tmp_data = [cluster_weight[train_c[_i]],y_pred[_i,1]]
    new_data.append(tmp_data)
new_data = np.array(new_data)

new_data_t = []
for _i in range(y_pred_t.shape[0]):
    tmp_data = [cluster_weight[test_c[_i]],y_pred_t[_i,1]]
    new_data_t.append(tmp_data)
new_data_t = np.array(new_data_t)

new_data_v = []
for _i in range(y_pred_v.shape[0]):
    tmp_data = [cluster_weight[valid_c[_i]],y_pred_v[_i,1]]
    new_data_v.append(tmp_data)
new_data_v = np.array(new_data_v)


mode = 2
if mode == 0:
    new_train_data = new_data
    new_train_label = Y_train.argmax(axis=1)

    new_test_data = new_data_t
    new_test_label = Y_test.argmax(axis=1)

    print("New_Test_Accuracy=>",np.mean((y_pred_t.argmax(axis=1)==new_test_label)))

if mode == 1:
    new_train_data = new_data_t[:1500,:]
    new_train_label = Y_test.argmax(axis=1)[:1500]

    new_test_data = new_data_t[1500:,:]
    new_test_label = Y_test.argmax(axis=1)[1500:]

    print("New_Test_Accuracy=>",np.mean((y_pred_t.argmax(axis=1)[1500:]==new_test_label)))

if mode == 2:
    new_train_data = new_data_v
    new_train_label = Y_val.argmax(axis=1)

    new_test_data = new_data_t
    new_test_label = Y_test.argmax(axis=1)

    print("New_Test_Accuracy=>",np.mean((y_pred_t.argmax(axis=1)==new_test_label)))


New_Test_Accuracy=> 0.21152328334648776


In [ ]:
from sklearn.metrics import confusion_matrix,f1_score
from sklearn.ensemble import RandomForestClassifier
clf1 = RandomForestClassifier(n_estimators=10, max_depth=None,
      min_samples_split=2, random_state=0)
clf1.fit(new_train_data, new_train_label)
y_pred = clf1.predict(new_test_data)
print("RandomForestClassifier ")
print('Confusion matrix(test):\n{}'.format(confusion_matrix(new_test_label, y_pred)))
print("Test_Accuracy=>",np.mean((y_pred==new_test_label)))
# print('f1-score=>(test):{}'.format(f1_score(new_test_label, y_pred)))


from sklearn.ensemble import ExtraTreesClassifier
clf2 = ExtraTreesClassifier(n_estimators=10, max_depth=None,
      min_samples_split=2, random_state=0)
clf2.fit(new_train_data, new_train_label)
y_pred = clf2.predict(new_test_data)
print("ExtraTreesClassifier")
print('Confusion matrix(test):\n{}'.format(confusion_matrix(new_test_label, y_pred)))
print("Test_Accuracy=>",np.mean((y_pred==new_test_label)))
# print('f1-score=>(test):{}'.format(f1_score(new_test_label, y_pred)))



from sklearn import svm
clf3 = svm.SVC()
clf3.fit(new_train_data, new_train_label)
y_pred = clf3.predict(new_test_data)
print("SVMClassifier")
print('Confusion matrix(test):\n{}'.format(confusion_matrix(new_test_label, y_pred)))
print("Test_Accuracy=>",np.mean((y_pred==new_test_label)))
# print('f1-score=>(test):{}'.format(f1_score(new_test_label, y_pred)))



from sklearn.neural_network import MLPClassifier
clf4 = MLPClassifier(random_state=1, max_iter=300).fit(new_train_data, new_train_label)
y_pred = clf4.predict(new_test_data)
print("MLPClassifier")
print('Confusion matrix(test):\n{}'.format(confusion_matrix(new_test_label, y_pred)))
print("Test_Accuracy=>",np.mean((y_pred==new_test_label)))
# print('f1-score=>(test):{}'.format(f1_score(new_test_label, y_pred)))


#Final Test

In [88]:
new_sentence_ori = ['Says the Annies List political group supports third-trimester abortions on demand.']
_emb_new_sentence = sentence_embedding(new_sentence_ori[0]).reshape(1, -1)
print("_emb_new_sentence.shape=>",_emb_new_sentence.shape)

new_sentence = norm_all_sent(new_sentence_ori)
print("1:new_sentence=>",new_sentence)
new_sentence = tokenizer.texts_to_sequences(new_sentence)
print("2:new_sentence=>",new_sentence)
new_sentence = pad_sequences(new_sentence , maxlen = max_seq_len)
print("3:new_sentence=>",new_sentence.shape)


from sklearn.metrics import euclidean_distances

_dist = euclidean_distances(_emb_new_sentence,all_emb)
_nearest_index = _dist.argmin()
_credit_rank = cluster_weight[train_c[_nearest_index]]
_credit_rank = _credit_rank.reshape(1)
print("_credit_rank.shape=>",_credit_rank.shape)

y_pred_new_sentence = model.predict([new_sentence,_credit_rank])
class_new_sentence = y_pred_new_sentence.argmax()
label_dict = ['false','pants-fire','half-true','mostly-true','barely-true','true']

print("******************")
print("******************")
print("Classificatin Results:")
for _i in range(len(label_dict)):
    print("Class:",label_dict[_i],"=> probability:",y_pred_new_sentence[0][_i])
print("******************")
print("******************")
print("Final Result for statement==>>", new_sentence_ori[0])
print("Class:",label_dict[class_new_sentence],"=> probability:",y_pred_new_sentence[0][class_new_sentence])

new_data_for_next_step = np.array([y_pred_new_sentence[0][class_new_sentence],_credit_rank[0]])
new_data_for_next_step = new_data_for_next_step.reshape(1, -1)


y_pred = clf3.predict(new_data_for_next_step)
print("Result of second stage of Classification:")
print("Predicted Label:",label_dict[y_pred[0]])


_emb_new_sentence.shape=> (1, 768)
1:new_sentence=> ['says the annies list political group supports thirdtrimester abortions on demand']
2:new_sentence=> [[7, 1, 6738, 1046, 481, 566, 352, 4897, 529, 11, 1479]]
3:new_sentence=> (1, 100)
_credit_rank.shape=> (1,)
******************
******************
Classificatin Results:
Class: false => probability: 0.18773924
Class: pants-fire => probability: 0.08303636
Class: half-true => probability: 0.2002887
Class: mostly-true => probability: 0.199299
Class: barely-true => probability: 0.16621561
Class: true => probability: 0.16342108
******************
******************
Final Result for statement==>> says the annies list political group supports thirdtrimester abortions on demand
Class: half-true => probability: 0.2002887
Result of second stage of Classification:
Predicted Label: half-true
